In [146]:
# import libraries
import numpy as np
import math


##Ejercicio 1

In [147]:
### a

# obtain the headers
with open('wine.csv', 'r') as file:
    headers = file.readline().split(',')

# read the data from the csv file
data = np.genfromtxt('wine.csv', delimiter=',')

# drop the text headers and keep only the values
data = data[1:]

In [148]:
### b

# get the independant variables
x = data[:, :-1]

# get the single dependant variable
y = data[:, -1]

In [149]:
### c

# obtain the mean vector of data
def get_mean_vector(x):
    return np.mean(x, axis=0)

# obtain the standard deviation vector of data
def get_std_vector(x):
    rows_qty = x.shape[0]
    col_qty = x.shape[1]
    ret = np.zeros(col_qty)

    mean = get_mean_vector(x)

    for i in range(rows_qty):
        ret = ret + (x[i] - mean) ** 2

    return ret / (rows_qty - 1)

# obtain the normalized vector of data
def get_normailized(x):
    return (x - get_mean_vector(x)) / get_std_vector(x)

x = get_normailized(x)

In [150]:
### d

cov = np.cov(x, rowvar=False)

In [151]:
### e

def get_max_eigenvector(matrix, iter = 20):
    # start with a random vector
    ret = np.random.rand(matrix.shape[1])

    # apply the power method, simply multiplying the matrix with the vector and dividing by the norm
    for _ in range(iter):
        next = np.dot(matrix, ret)
        ret = next / np.linalg.norm(next)
    return ret

# obtain the eigenvalue simply by calculating using the formula see in class
def get_related_eigenvalue(matrix, max_eigenvector):
    t_max_eigenvector = np.transpose(max_eigenvector)
    return np.dot(t_max_eigenvector, np.dot(matrix, max_eigenvector)) / (np.dot(t_max_eigenvector, max_eigenvector))

max_cov_eigenvector = get_max_eigenvector(cov)
max_cov_eigenvalue = get_related_eigenvalue(cov, max_cov_eigenvector)


In [154]:
### f

def get_max_eigenvectors(matrix, qty = 1, iter = 200):
    
    eigenvectors = np.zeros(shape=(qty, matrix.shape[1]))

    for i in range(qty):
        aux = np.random.rand(matrix.shape[1])

        # for the next iteration start with a random vector and remove the components of already calculed eigenvectors
        for _ in range(iter):
            for eigenvector in eigenvectors:
                aux = aux - np.dot(eigenvector, aux) * eigenvector
            next_aux = np.dot(matrix, aux)
            aux = next_aux / np.linalg.norm(next_aux)

        eigenvectors[i] = aux

    return eigenvectors

def get_max_eigenvalues(matrix, eigenvectors):
    eigenvalues = np.zeros(eigenvectors.shape[0])

    i = 0
    for eigenvector in eigenvectors:
        eigenvalues[i] = get_related_eigenvalue(matrix, eigenvector)
        i = i + 1
        
    return eigenvalues

eigenvectors = get_max_eigenvectors(cov, cov.shape[0])
eigenvalues = np.array(get_max_eigenvalues(cov, eigenvectors))

# print(np.allclose(np.linalg.eig(cov).eigenvalues, eigenvalues, rtol=1e-03))
# print(np.allclose(np.transpose(np.linalg.eig(cov).eigenvectors  ** 2), eigenvectors  ** 2, rtol=1e-03))